In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [13]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [16]:
y_pred.std()

6.247488852238703

In [19]:
df['ride_id'] = f'{2023:04d}/{3:02d}_' + df.index.astype('str')

In [40]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred


In [29]:
df_2 = pd.DataFrame(y_pred, columns=['y_pred'])

In [30]:
df_result = pd.concat([df, df_2], axis = 1)

In [34]:
output_file = f'output/{2023:04d}-{3:02d}.parquet'

In [41]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)